## consolidate predictions detail and get global metrics

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.layouts import row, gridplot, layout
from bokeh.palettes import d3
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [3]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [4]:
pd.set_option('display.max_rows', 200)

In [5]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [6]:
data_folder = '{}/datasets/traffic/PEMS-SF'.format(PROJECT_ROOT)
data_folder

'/home/developer/gcp/cbidmltsf/datasets/traffic/PEMS-SF'

In [7]:
# build the station_ids list

In [8]:
def process_list(s, variable_type=int, delimiter=None):
    """Parses a line in the PEMS format to a list."""
    if delimiter is None:
      l = [
          variable_type(i) for i in s.replace('[', '').replace(']', '').split()
      ]
    else:
      l = [
          variable_type(i)
          for i in s.replace('[', '').replace(']', '').split(delimiter)
      ]

    return l

In [9]:
def read_single_list(filename):
    """Returns single list from a file in the PEMS-custom format."""
    with open(os.path.join(data_folder, filename), 'r') as dat:
        l = process_list(dat.readlines()[0])
    return l

In [10]:
station_ids = [id for id in read_single_list('stations_list')]
len(station_ids)

963

In [11]:
model_ids = [
    # execution 0 at 5K, 10K, 15K, 20K, 25K train_steps
    'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075',
    # execution 1 at 5K, 10K, 15K, 20K, 25K, 30K, 35K train_steps
    'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075',
    'BSCTRFM_TPU_075', 'BSCTRFM_TPU_075'
]

In [12]:
executions = [
    # execution 0 at 5K, 10K, 15K, 20K, 25K train_steps
    0, 0, 0, 0, 0,
    # execution 1 at 5K, 10K, 15K, 20K, 25K, 30K, 35K train_steps
    1, 1, 1, 1, 1, 1, 1
]

In [13]:
saved_model_ids = [
    # execution 0 at 5K, 10K, 15K, 20K, 25K train_steps
    '1637776739', '1637777357', '1637778389', '1637778735', '1637780410',
    # execution 1 at 5K, 10K, 15K, 20K, 25K, 30K, 35K train_steps
    '1637777047', '1637777674', '1637778065', '1637779049', '1637779397', '1637779753', '1637780062'
]

In [14]:
# build a dictionary to recover training information by saved model
training_info = {
    '1637776739': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 0,
        'train_steps': 5000
    },
    '1637777357': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 0,
        'train_steps': 10000
    },
    '1637778389': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 0,
        'train_steps': 15000
    },
    '1637778735': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 0,
        'train_steps': 20000
    },
    '1637780410': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 0,
        'train_steps': 25000
    },
    '1637777047': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 5000
    },
    '1637777674': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 10000
    },
    '1637778065': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 15000
    },
    '1637779049': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 20000
    },
    '1637779397': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 25000
    },
    '1637779753': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 30000
    },
    '1637780062': {
        'model_id': 'BSCTRFM_TPU_075',
        'execution': 1,
        'train_steps': 35000
    }
}

In [15]:
for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):
    print(model_id, execution, saved_model_id)

BSCTRFM_TPU_075 0 1637776739
BSCTRFM_TPU_075 0 1637777357
BSCTRFM_TPU_075 0 1637778389
BSCTRFM_TPU_075 0 1637778735
BSCTRFM_TPU_075 0 1637780410
BSCTRFM_TPU_075 1 1637777047
BSCTRFM_TPU_075 1 1637777674
BSCTRFM_TPU_075 1 1637778065
BSCTRFM_TPU_075 1 1637779049
BSCTRFM_TPU_075 1 1637779397
BSCTRFM_TPU_075 1 1637779753
BSCTRFM_TPU_075 1 1637780062


In [16]:
sldb_id = 'PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'
sldb_id

'PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [17]:
forecast_window = 24
inference = '{:03d}'.format(forecast_window)

In [18]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)
data_dir

'/home/developer/gcp/cbidmltsf/sldbs/PEMS-SF_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [19]:
# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [20]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [21]:
ts_identifier = sldb_dict['ts']
m = sldb_dict['embedding']['hourly']
t = sldb_dict['no_targets']

In [22]:
ts_identifier, m, t

('PEMS-SF_SEPARATED_FULL', 168, 168)

## skip the next code cells if predictions detail dataframe is already persisted!

In [23]:
predictions_detail_columns=[
    'model_id', 'execution', 'saved_model_id',
    'inference', 'event',
    'station_id', 'ts_index', 'prediction', 'target'
]

In [24]:
predictions_detail_df = pd.DataFrame(columns=predictions_detail_columns)

In [38]:
# inference event value can be retrieved from the pickle filename
# define it here as only inference event zero has been run

event = 0

# iterate over model_id, execution, saved_model_id combinations
for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):
    
    print('Processing predictions for model {}, execution {}, saved model {}...'.\
          format(model_id, execution, saved_model_id)
         )
    
    # for each saved_model_id, iterate over persisted predictions for the test dataset
    for station_id in station_ids:
        detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
            PROJECT_ROOT,
            'database',
            'predictions_detail',
            model_id,
            execution,
            saved_model_id,
            # for electricity dataset, replace dataset with customer_id
            station_id,
            inference,
            event)
        
        loaded_predictions_df = pd.read_pickle(detail_pickle_path)
        
        # pass info on the 7-row loaded_predictions_df
        # to a 168-row predictions_detail_df, use a buffer_df

        # for each customer, iterate on starting rows for each day
        for index, row in loaded_predictions_df.iterrows():

            # make a buffer dataframe for one-day predictions
            buffer_df = pd.DataFrame()

            # populate the buffer dataframe

            buffer_df['model_id'] = forecast_window*[model_id]
            buffer_df['execution'] = forecast_window*[execution]
            buffer_df['saved_model_id'] = forecast_window*[saved_model_id]
            buffer_df['station_id'] = forecast_window*[station_id]
            buffer_df['inference'] = forecast_window*[inference]
            buffer_df['event'] = forecast_window*[event]

            buffer_df['ts_index'] = row['string_indexes']
            buffer_df['prediction'] = row['predictions']
            buffer_df['target'] =  row['targets']

            predictions_detail_df = pd.concat(
                [predictions_detail_df, buffer_df])
            


# once added all requested model_id/execution/saved_model_id combinations
# and all 7 24-h rows to the predictions detail dataframe,
# for all the customer_ids, reset the index
predictions_detail_df = predictions_detail_df.reset_index()



Processing predictions for model BSCTRFM_TPU_075, execution 0, saved model 1637776739...
Processing predictions for model BSCTRFM_TPU_075, execution 0, saved model 1637777357...
Processing predictions for model BSCTRFM_TPU_075, execution 0, saved model 1637778389...
Processing predictions for model BSCTRFM_TPU_075, execution 0, saved model 1637778735...
Processing predictions for model BSCTRFM_TPU_075, execution 0, saved model 1637780410...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637777047...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637777674...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637778065...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637779049...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637779397...
Processing predictions for model BSCTRFM_TPU_075, execution 1, saved model 1637779753...
Processing prediction

In [39]:
# ToDo: persist the predictions_detail_df in /databases

In [40]:
predictions_detail_df.to_pickle('/home/developer/gcp/cbidmltsf/database/traffic.pkl')

In [25]:
# just load the predictions detail dataframe (previously persisted)

In [26]:
predictions_detail_df = pd.read_pickle('/home/developer/gcp/cbidmltsf/database/traffic.pkl')

In [27]:
predictions_detail_df

,index,model_id,execution,saved_model_id,inference,event,station_id,ts_index,prediction,target
0,0,BSCTRFM_TPU_075,0,1637776739,024,0,400000,3983,0.011851,0.010100
1,1,BSCTRFM_TPU_075,0,1637776739,024,0,400000,3984,0.011482,0.008633
2,2,BSCTRFM_TPU_075,0,1637776739,024,0,400000,3985,0.012259,0.009250
3,3,BSCTRFM_TPU_075,0,1637776739,024,0,400000,3986,0.015281,0.010917
4,4,BSCTRFM_TPU_075,0,1637776739,024,0,400000,3987,0.029714,0.019500
...,...,...,...,...,...,...,...,...,...,...
1941403,19,BSCTRFM_TPU_075,1,1637780062,024,0,402090,10160942,0.043358,0.041400
1941404,20,BSCTRFM_TPU_075,1,1637780062,024,0,402090,10160943,0.034047,0.037300
1941405,21,BSCTRFM_TPU_075,1,1637780062,024,0,402090,10160944,0.034439,0.035400
1941406,22,BSCTRFM_TPU_075,1,1637780062,024,0,402090,10160945,0.030628,0.029750


In [28]:
# get statistics from the predictions detail dataframe by saved model identifier

# a dictionary of dictionaries to store inference metrics
metrics = dict()

for saved_model_id in saved_model_ids:
    
    # a dictionary for metrics on the current saved model
    metrics[saved_model_id] = dict()
    
    results_df = predictions_detail_df[predictions_detail_df["saved_model_id"] == saved_model_id]
    
    metrics[saved_model_id]['mae'] = mean_absolute_error(
        results_df['prediction'], results_df['target']
    )
    
    true_values_average = np.mean(
        results_df['target']
    )
    
    metrics[saved_model_id]['nd'] = metrics[saved_model_id]['mae']/true_values_average

    metrics[saved_model_id]['rmse'] = sqrt(
        mean_squared_error(
            results_df['prediction'], results_df['target']
        )
    )
    
    metrics[saved_model_id]['nrmse'] = metrics[saved_model_id]['rmse']/true_values_average

    metrics[saved_model_id]['smape'] = symmetric_mean_absolute_percentage_error(
        results_df['prediction'], results_df['target']
    )

    print('Saved model {}, ND: {}, NRMSE: {}, SMAPE: {}'.format(saved_model_id,
                                                                metrics[saved_model_id]['nd'],
                                                                metrics[saved_model_id]['nrmse'],
                                                                metrics[saved_model_id]['smape']
                                                               )
         )

Saved model 1637776739, ND: 0.15645542563251502, NRMSE: 0.4117894286600512, SMAPE: 0.17107462776163906
Saved model 1637777357, ND: 0.1545584604720366, NRMSE: 0.41035559298262697, SMAPE: 0.1678727068085735
Saved model 1637778389, ND: 0.14175983032321485, NRMSE: 0.4086718795223467, SMAPE: 0.13940278160305458
Saved model 1637778735, ND: 0.20298197165304707, NRMSE: 0.4459896784061798, SMAPE: 0.2095254099378012
Saved model 1637780410, ND: 0.1663874557950333, NRMSE: 0.42501984120170405, SMAPE: 0.15919810580022342
Saved model 1637777047, ND: 0.1835924421126416, NRMSE: 0.4310319566047621, SMAPE: 0.20602659815734223
Saved model 1637777674, ND: 0.1555171332446303, NRMSE: 0.406064848810865, SMAPE: 0.15362525013528225
Saved model 1637778065, ND: 0.1779801622294446, NRMSE: 0.4195762598379989, SMAPE: 0.18755177672445308
Saved model 1637779049, ND: 0.1922904023470717, NRMSE: 0.433924791112982, SMAPE: 0.20326514934441162
Saved model 1637779397, ND: 0.17705422993164913, NRMSE: 0.4414993054749113, SMAPE

In [29]:
# merge training info and metrics dictionaries to a dataframe

In [30]:
training_info_df = pd.DataFrame.from_dict(training_info, orient='index')
training_info_df

,model_id,execution,train_steps
1637776739,BSCTRFM_TPU_075,0,5000
1637777357,BSCTRFM_TPU_075,0,10000
1637778389,BSCTRFM_TPU_075,0,15000
1637778735,BSCTRFM_TPU_075,0,20000
1637780410,BSCTRFM_TPU_075,0,25000
1637777047,BSCTRFM_TPU_075,1,5000
1637777674,BSCTRFM_TPU_075,1,10000
1637778065,BSCTRFM_TPU_075,1,15000
1637779049,BSCTRFM_TPU_075,1,20000
1637779397,BSCTRFM_TPU_075,1,25000


In [31]:
metrics_df = pd.DataFrame.from_dict(metrics, orient='index')
metrics_df

,mae,nd,rmse,nrmse,smape
1637776739,0.008185,0.156455,0.021543,0.411789,0.171075
1637777357,0.008086,0.154558,0.021468,0.410356,0.167873
1637778389,0.007416,0.141760,0.021379,0.408672,0.139403
1637778735,0.010619,0.202982,0.023332,0.445990,0.209525
1637780410,0.008704,0.166387,0.022235,0.425020,0.159198
1637777047,0.009605,0.183592,0.022549,0.431032,0.206027
1637777674,0.008136,0.155517,0.021243,0.406065,0.153625
1637778065,0.009311,0.177980,0.021950,0.419576,0.187552
1637779049,0.010060,0.192290,0.022701,0.433925,0.203265
1637779397,0.009263,0.177054,0.023097,0.441499,0.166299


In [32]:
predictions_summary_df = pd.concat([training_info_df, metrics_df], axis=1)
predictions_summary_df

,model_id,execution,train_steps,mae,nd,rmse,nrmse,smape
1637776739,BSCTRFM_TPU_075,0,5000,0.008185,0.156455,0.021543,0.411789,0.171075
1637777357,BSCTRFM_TPU_075,0,10000,0.008086,0.154558,0.021468,0.410356,0.167873
1637778389,BSCTRFM_TPU_075,0,15000,0.007416,0.141760,0.021379,0.408672,0.139403
1637778735,BSCTRFM_TPU_075,0,20000,0.010619,0.202982,0.023332,0.445990,0.209525
1637780410,BSCTRFM_TPU_075,0,25000,0.008704,0.166387,0.022235,0.425020,0.159198
1637777047,BSCTRFM_TPU_075,1,5000,0.009605,0.183592,0.022549,0.431032,0.206027
1637777674,BSCTRFM_TPU_075,1,10000,0.008136,0.155517,0.021243,0.406065,0.153625
1637778065,BSCTRFM_TPU_075,1,15000,0.009311,0.177980,0.021950,0.419576,0.187552
1637779049,BSCTRFM_TPU_075,1,20000,0.010060,0.192290,0.022701,0.433925,0.203265
1637779397,BSCTRFM_TPU_075,1,25000,0.009263,0.177054,0.023097,0.441499,0.166299


In [33]:
p = figure(title='For Traffic/ND train under 15K steps', x_axis_label='Training steps', y_axis_label='ND')

objective = Span(location=0.17,
                 dimension='width',
                 line_color='red',
                 line_dash='dashed',
                 line_width=2)

p.add_layout(objective)

p.line(
    x=predictions_summary_df['1637776739':'1637780410'].train_steps,
    y=predictions_summary_df['1637776739':'1637780410'].nd,
    color='blue',
    legend_label='075_00'
)

p.line(
    x=predictions_summary_df['1637777047':'1637780062'].train_steps,
    y=predictions_summary_df['1637777047':'1637780062'].nd,
    color='orange',
    legend_label='075_01'
)

show(p)

In [34]:
p = figure(title='For Traffic/NRMSE train under 15K steps', x_axis_label='Training steps', y_axis_label='NRMSE')

objective = Span(location=0.42,
                 dimension='width',
                 line_color='red',
                 line_dash='dashed',
                 line_width=2)

p.add_layout(objective)

p.line(
    x=predictions_summary_df['1637776739':'1637780410'].train_steps,
    y=predictions_summary_df['1637776739':'1637780410'].nrmse,
    color='blue',
    legend_label='075_00'
)

p.line(
    x=predictions_summary_df['1637777047':'1637780062'].train_steps,
    y=predictions_summary_df['1637777047':'1637780062'].nrmse,
    color='orange',
    legend_label='055_01'
)

show(p)